from IPython.display import display, Math
display(Math(r'Dims: {}x{}m \\ Area: {}m^2 \\ Volume: {}m^3'.format(a, round(b,2), P, V)))

Numpy package used to store and manipulate data

In [1]:
import numpy as np

Using a numpy array to represent qubits, fast and manipulable. where:

$$qubit(a,b)  = \frac{a}{\sqrt{|a|^{2}+|b|^{2}}}|0\rangle+\frac{a}{\sqrt{|a|^{2}+|b|^{2}}}|1\rangle\$$



In [2]:
def qubit(a,b):                          #function creates a normed vector with 2 entries
    Length = np.sqrt(np.abs(a)**2+np.abs(b)**2)
    return(np.array([a/Length,b/Length]))

In [3]:
def qudit(*args):                         #function creates a normed vector with arbitrary entries
    Length = sum([i**2 for i in args])**(1/len(args))
    return(np.array([j/Length for j in args]))

Want to generate a random number uniformly between 0 and 1 to simulate measurements.  
remove 0s  
permute all quotients of pairs of coefficients, take the maximum.  
1/max determines how finely to generate.

Gates are represented using numpys matrix functions. The function below accepts four parameters for each of the entries in a 2x2 matrix and one additional coefficent parameter to scale all the individual entries by. 

$$gate(a\ ,b\ ,c\ ,d\ ,\mathit{coefficient}\ ) = \mathit{coefficient}*\begin{pmatrix} a& b\\ c&d \end{pmatrix} $$

The coefficient parameter is 1 by default and is useful for representing gates like the hadamard gate as shown below:

$$gate(1\ ,1\ ,1\ ,-1\ ,2**(-0.5)\ ) = \frac{1}{\sqrt{2}}\begin{pmatrix} 1& 1\\  1&-1 \end{pmatrix} $$

In [4]:
def gate(a ,b ,c ,d ,coefficient = 1):
    matrix = np.matrix([[a,b],[c,d]])*coefficient                            #forms the matrix
    conj_trans_matrix = (coefficient*np.matrix([[a,c],[b,d]])).conj()        #forms the conjugate transpose of the matrix
    if np.allclose(matrix*conj_trans_matrix, np.matrix([[1.,0.],[0.,1.]])):  #Checks if the matrix is unitary
        return(matrix)
    raise Exception('Coefficients do not form a unitary matrix')
    
#later: if the matrix formed is some factor off of being unitary then automatically convert it.
#np.allclose only compares each element within tolerance

Assigned variable names to some commonly used gates below for ease of use.

Also serves to test the 'gate' function defined above, no errors raised means all the gates are unitary as expected

In [5]:
##Some commonly used gates are initialised below
I     = gate(1,0,0,1)            #The identity gate
Hgate = gate(1,1,1,-1,2**(-0.5)) # Hadamard gate
Xgate = gate(0,1,1,0)           # Pauli-X gate
Ygate = gate(0,-1j,1j,0)         # Pauli-Y gate
Zgate = gate(1,0,0,-1)          # Pauli-Z gate
Sgate = gate(1,0,0,1j)          # Phase gate
Tgate = gate(1,0,0,1j**0.5)     # Pi/8 gate

print(np.allclose(Xgate*Ygate*Xgate,-Ygate)) # verification that XYX = -Y

True


Naturally numpy allows multiplication of vectors and matrices easily and one can see below that after applying the Hadamard gate to the initial state:

$$1|0\rangle+0|1\rangle$$ 
one obtains the superposition:

$$\frac{1}{\sqrt{2}}|0\rangle+\frac{1}{\sqrt{2}}|1\rangle\$$

In [6]:
print(qubit(1,0)*Hgate)

[[0.70710678 0.70710678]]


In [7]:
def Cgate(gate):          #forms a controlled gate(4x4) from a regular gate(2x2).
    row1 = [1,0,0,0]
    row2 = [0,1,0,0]
    row3 = [0,0,gate[0,0],gate[0,1]]
    row4 = [0,0,gate[1,0],gate[1,1]]
    return(np.matrix([row1,row2,row3,row4]))

In [24]:
row1 = [0,1,0,0]
row2 = [1,0,0,0]
row3 = [0,0,1,0]
row4 = [0,0,0,1]
ton_dell= (np.matrix([row1,row2,row3,row4]))

Cgate(Xgate)*ton_dell*Cgate(Xgate)

matrix([[0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])

In [9]:
Cgate(Xgate)

matrix([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0]])

In [10]:
np.kron(I,Xgate)

matrix([[0, 1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0]])

In [11]:
q0 = qubit(1,0)
q1 = qubit(0,1)

In [12]:
Statevector = np.kron(q1,q1)

print(Statevector)

print(Statevector*Cgate(Xgate))

print(np.kron(q1,q0))

[0. 0. 0. 1.]
[[0. 0. 1. 0.]]
[0. 0. 1. 0.]


In [13]:
Statevector*Cgate(Xgate)

matrix([[0., 0., 1., 0.]])

# Deutsch's algorithm

first prepare the input state: $|\psi_0\rangle = |01\rangle = (1,0) \oplus (0,1)$

In [18]:
Psi0 = np.kron(qubit(1,0),qubit(0,1))
print(Psi0)

[0. 1. 0. 0.]


Apply the Hadamard gate to each qubit individually  
$ |\psi_1\rangle = |\psi_0\rangle \cdot( H\oplus H) $

In [20]:
Psi1 = Psi0*np.kron(Hgate,Hgate)
print(Psi1)

[[ 0.5 -0.5  0.5 -0.5]]


In [21]:
np.kron(q0*Hgate,q1*Hgate) #same as first applying the Hadamard gate then taking the kronecker delta

matrix([[ 0.5, -0.5,  0.5, -0.5]])

Apply the Hadamard gate to the first qubit only:  
$ |\psi_3\rangle = |\psi_2\rangle \cdot( H\oplus I)$